![MLU Logo](https://drive.corp.amazon.com/view/bwernes@/MLU_Logo.png?download=true)

# NLP2 Lecture 4 Support Notebook

# Fine-tuning Sentence Pair Classification with BERT
Pre-trained language representations have been shown to improve many downstream NLP tasks such as question answering, and natural language inference. To apply pre-trained representations to these tasks, there are two main strategies:

The feature-based approach, which uses the pre-trained representations as additional features to the downstream task. 

Or the fine-tuning-based approach, which trains the downstream tasks by fine-tuning pre-trained parameters. 

While feature-based approaches such as ELMo [3] (introduced in the previous tutorial) are effective in improving many downstream tasks, they require task-specific architectures. Devlin, Jacob, et al proposed BERT [1] (Bidirectional Encoder Representations from Transformers), which fine-tunes deep bi-directional representations on a wide range of tasks with minimal task-specific parameters, and obtains state- of-the-art results.

In this tutorial, we will focus on fine-tuning with the pre-trained BERT model to classify semantically equivalent sentence pairs.

Specifically, we will:

1. Load the state-of-the-art pre-trained BERT model and attach an additional layer for classification
2. Process and transform sentence-pair data for the task at hand
3. Fine-tune the BERT model for sentence classification

In [1]:
!pip install gluonnlp
!pip install mxnet==1.6.0
!pip install -q -U --pre mxnet-cu101mkl  # updating mxnet to at least v1.6

     |████████████████████████████████| 344 kB 15.6 MB/s eta 0:00:01
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=472988 sha256=9e3e68ab5deeb18ed39602a31a12d0d5072b6576dd4931d398c7ae27c9526403
  Stored in directory: /home/ec2-user/.cache/pip/wheels/62/62/9a/53be069ac8c9dde533dacce0e716193a4a43e87b5d37f5008c
Successfully built gluonnlp
     |████████████████████████████████| 68.7 MB 71.1 MB/s eta 0:00:01     |████████████████████████▎       | 52.1 MB 19.1 MB/s eta 0:00:01


In [2]:
!pip install bert

  Created wheel for bert: filename=bert-2.2.0-py3-none-any.whl size=3753 sha256=1ac62d818e9d4f71e6353cdce2dc0b791b6c715478d69a77ac15541eee995abc
  Stored in directory: /home/ec2-user/.cache/pip/wheels/cb/da/90/bc3128c1676721ede415ed707464c4ea5671ee3e292e6b5404
  Created wheel for erlastic: filename=erlastic-2.0.0-py3-none-any.whl size=6788 sha256=5fd466d7c879a870c05cdb8183197e59ff6d6035fe7aca08e873f27eeedce769
  Stored in directory: /home/ec2-user/.cache/pip/wheels/10/bb/e1/a8f26a46f7f4d9278a50c924b93a1664e95db12b0d785da5b4
Successfully built bert erlastic


## Importing necessary modules

In [3]:
import warnings
warnings.filterwarnings('ignore')

import io
import random
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from gluonnlp.calibration import BertLayerCollector
# this notebook assumes that all required scripts are already
# downloaded from the corresponding tutorial webpage on http://gluon-nlp.mxnet.io
from bert import *

nlp.utils.check_version('0.8.1')
print("MXnet version: {}".format(mx.__version__))

MXnet version: 1.6.0


## Setting up the environment

Please note the comment in the code if no GPU is available.

In [4]:
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
# change `ctx` to `mx.cpu()` if no GPU is available.
ctx = mx.gpu(0)

## Using the pre-trained BERT model
In this demo, the BERT model we will use is BERT BASE trained on an uncased corpus of books and the English Wikipedia dataset in the GluonNLP model zoo.

## Get BERT

Let’s first take a look at the BERT model architecture for sentence pair classification below:

<img src="../images/bert-sentence-pair.png" alt="drawing" width="400"/>

where the model takes a pair of sequences and pools the representation of the first token in the sequence. <br/>
Note that the original BERT model was trained for a masked language model and next-sentence prediction tasks, which includes layers for language model decoding and classification. These layers will not be used for fine-tuning the sentence pair classification.

We can load the pre-trained BERT fairly easily using the model API in GluonNLP, which returns the vocabulary along with the model. We include the pooler layer of the pre-trained model by setting use_pooler to True.

In [5]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                                             dataset_name='book_corpus_wiki_en_uncased',
                                             pretrained=True, ctx=ctx, use_pooler=True,
                                             use_decoder=False, use_classifier=False)
print(bert_base)

Vocab file is not found. Downloading.
BERTModel(
  (encoder): BERTEncoder(
    (dropout_layer): Dropout(p = 0.1, axes=())
    (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
    (transformer_cells): HybridSequential(
      (0): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (attention_cell): DotProductSelfAttentionCell(
          (dropout_layer): Dropout(p = 0.1, axes=())
        )
        (proj): Dense(768 -> 768, linear)
        (ffn): PositionwiseFFN(
          (ffn_1): Dense(768 -> 3072, linear)
          (activation): GELU()
          (ffn_2): Dense(3072 -> 768, linear)
          (dropout_layer): Dropout(p = 0.1, axes=())
          (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
        )
        (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
      )
      (1): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (at

## Transform the model for SentencePair classification

Now that we have loaded the BERT model, we only need to attach an additional layer for classification. The BERTClassifier class uses a BERT base model to encode sentence representation, followed by a nn.Dense layer for classification.

In [6]:
bert_classifier = nlp.model.BERTClassifier(bert_base, num_classes=2, dropout=0.1)
# only need to initialize the classifier layer.
bert_classifier.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
bert_classifier.hybridize(static_alloc=True)

# softmax cross entropy loss for classification
loss_function = mx.gluon.loss.SoftmaxCELoss()
loss_function.hybridize(static_alloc=True)

metric = mx.metric.Accuracy()

## Data preprocessing for BERT

For this demo, we need to do a bit of preprocessing before feeding our data introduced the BERT model. Here we want to leverage the dataset included in the downloaded archive at the beginning of this tutorial.

## Loading the dataset

We use the dev set of the Microsoft Research Paraphrase Corpus dataset. The file is named ‘dev.tsv’. Let’s take a look at the first few lines of the raw dataset.

In [10]:
# import the datasets
import boto3
import os.path
from os import path
bucketname = 'mlu-courses-datalake' # replace with your bucket name
filename = 'NLP2/data/dev.tsv' # replace with your object key
local_filename = "./dev.tsv" # replace with your object key
s3 = boto3.resource('s3')
if not path.exists(local_filename):
    s3.Bucket(bucketname).download_file(filename, local_filename)

The file contains 5 columns, separated by tabs. The header of the file explains each of these columns, although an explanation for each is included here: 0. The label indicating whether the two sentences are semantically equivalent 1. The id of the first sentence in this sample 2. The id of the second sentence in this sample 3. The content of the first sentence 4. The content of the second sentence

For our task, we are interested in the 0th, 3rd and 4th columns. To load this dataset, we can use the TSVDataset API and skip the first line because it’s just the schema:

In [11]:
# Skip the first line, which is the schema
num_discard_samples = 1
# Split fields by tabs
field_separator = nlp.data.Splitter('\t')
# Fields to select from the file
field_indices = [3, 4, 0]
data_train_raw = nlp.data.TSVDataset(filename=local_filename,
                                 field_separator=field_separator,
                                 num_discard_samples=num_discard_samples,
                                 field_indices=field_indices)
sample_id = 0
# Sentence A
print(data_train_raw[sample_id][0])
# Sentence B
print(data_train_raw[sample_id][1])
# 1 means equivalent, 0 means not equivalent
print(data_train_raw[sample_id][2])

He said the foodservice pie business doesn 't fit the company 's long-term growth strategy .
" The foodservice pie business does not fit our long-term growth strategy .
1


To use the pre-trained BERT model, we need to pre-process the data in the same way it was trained. The following figure shows the input representation in BERT:

<img src="../images/bert-embed.png" alt="drawing" width="400"/>

We will use BERTDatasetTransform to perform the following transformations: - tokenize the input sequences - insert [CLS] at the beginning - insert [SEP] between sentence A and sentence B, and at the end - generate segment ids to indicate whether a token belongs to the first sequence or the second sequence. - generate valid length

In [13]:
import bert
# Use the vocabulary from pre-trained model for tokenization
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)

# The maximum length of an input sequence
max_len = 128

# The labels for the two classes [(0 = not similar) or  (1 = similar)]
all_labels = ["0", "1"]

# whether to transform the data as sentence pairs.
# for single sentence classification, set pair=False
# for regression task, set class_labels=None
# for inference without label available, set has_label=False
pair = True
transform = bert.data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                                                class_labels=all_labels,
                                                has_label=True,
                                                pad=True,
                                                pair=pair)
data_train = data_train_raw.transform(transform)

print('vocabulary used for tokenization = \n%s'%vocabulary)
print('%s token id = %s'%(vocabulary.padding_token, vocabulary[vocabulary.padding_token]))
print('%s token id = %s'%(vocabulary.cls_token, vocabulary[vocabulary.cls_token]))
print('%s token id = %s'%(vocabulary.sep_token, vocabulary[vocabulary.sep_token]))
print('token ids = \n%s'%data_train[sample_id][0])
print('segment ids = \n%s'%data_train[sample_id][1])
print('valid length = \n%s'%data_train[sample_id][2])
print('label = \n%s'%data_train[sample_id][3])

AttributeError: module 'bert' has no attribute 'transform'

## Fine-tuning the model

Now we have all the pieces to put together, and we can finally start fine-tuning the model with very few epochs. For demonstration, we use a fixed learning rate and skip the validation steps. For the optimizer, we leverage the ADAM optimizer which performs very well for NLP data and for BERT models in particular.

In [11]:
%%time
# around 00:48 minutes with gpu
# The hyperparameters
batch_size = 32
lr = 5e-6

# The FixedBucketSampler and the DataLoader for making the mini-batches
train_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[2]) for item in data_train],
                                            batch_size=batch_size,
                                            shuffle=True)
bert_dataloader = mx.gluon.data.DataLoader(data_train, batch_sampler=train_sampler)

trainer = mx.gluon.Trainer(bert_classifier.collect_params(), 'adam',
                           {'learning_rate': lr, 'epsilon': 1e-9})

# Collect all differentiable parameters
# `grad_req == 'null'` indicates no gradients are calculated (e.g. constant parameters)
# The gradients for these params are clipped later
params = [p for p in bert_classifier.collect_params().values() if p.grad_req != 'null']
grad_clip = 1

# Training the model with only three epochs
log_interval = 4
num_epochs = 3
for epoch_id in range(num_epochs):
    metric.reset()
    step_loss = 0
    for batch_id, (token_ids, segment_ids, valid_length, label) in enumerate(bert_dataloader):
        with mx.autograd.record():

            # Load the data to the GPU
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # Forward computation
            ############################# ATTENTION ###############################################################
            # Intermitent ERROR here!!! Maybe needs to restart kernel after running previous demo in this notebook
            ############################# ATTENTION ###############################################################
            out = bert_classifier(token_ids.as_nd_ndarray(), segment_ids, valid_length.astype('float32'))
            ls = loss_function(out, label).mean()

        # And backwards computation
        ls.backward()

        # Gradient clipping
        trainer.allreduce_grads()
        nlp.utils.clip_grad_global_norm(params, 1)
        trainer.update(1)

        step_loss += ls.asscalar()
        metric.update([label], [out])

        # Printing vital information
        if (batch_id + 1) % (log_interval) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f}'
                         .format(epoch_id, batch_id + 1, len(bert_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
            step_loss = 0

[Epoch 0 Batch 4/18] loss=0.7472, lr=0.0000050, acc=0.354
[Epoch 0 Batch 8/18] loss=0.7646, lr=0.0000050, acc=0.358
[Epoch 0 Batch 12/18] loss=0.7310, lr=0.0000050, acc=0.381
[Epoch 0 Batch 16/18] loss=0.7250, lr=0.0000050, acc=0.389
[Epoch 1 Batch 4/18] loss=0.6468, lr=0.0000050, acc=0.633
[Epoch 1 Batch 8/18] loss=0.6667, lr=0.0000050, acc=0.608
[Epoch 1 Batch 12/18] loss=0.6878, lr=0.0000050, acc=0.580
[Epoch 1 Batch 16/18] loss=0.6746, lr=0.0000050, acc=0.579
[Epoch 2 Batch 4/18] loss=0.6353, lr=0.0000050, acc=0.660
[Epoch 2 Batch 8/18] loss=0.5815, lr=0.0000050, acc=0.676
[Epoch 2 Batch 12/18] loss=0.5529, lr=0.0000050, acc=0.686
[Epoch 2 Batch 16/18] loss=0.6366, lr=0.0000050, acc=0.672
CPU times: user 36.1 s, sys: 13.3 s, total: 49.4 s
Wall time: 48.2 s


## Conclusion

In this tutorial, we showed how to fine-tune a sentence pair classification model with pre-trained BERT parameters. In GluonNLP, this can be done with such few, simple steps. All we did was apply a BERT-style data transformation to pre-process the data, automatically download the pre-trained model, and feed the transformed data into the model, all within 50 lines of code!

For demonstration purpose, we skipped the warmup learning rate schedule and validation on the dev dataset used in the original implementation. Please visit the BERT model zoo webpage, or the scripts/bert folder in the Github repository for the complete fine-tuning scripts.

## End of Example.  Return to Slides

![MLU Logo](https://drive.corp.amazon.com/view/bwernes@/MLU_Logo.png?download=true)

<div class="lev1">
    <a href="#NLP2-Lecture-4-Support-Notebook">
        <span class="toc-item-num">&nbsp;&nbsp;</span>
        Go to TOP
    </a>
</div>